In [7]:
# 🚀 AI Social Media Manager – Pro Chat Edition (STABLE)

import subprocess, sys
import gradio as gr
from groq import Groq
from google.colab import userdata

# ---------------- INSTALL ----------------
def install():
    try:
        import groq, gradio
    except:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "groq", "gradio"])

install()

# ---------------- API KEY ----------------
GROQ_API_KEY = userdata.get("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("❌ GROQ_API_KEY not found in Colab Secrets")

client = Groq(api_key=GROQ_API_KEY)
MODEL = "llama-3.3-70b-versatile"

# ---------------- PROMPT BUILDER ----------------
def build_system_prompt(platform, tone, language, goal, length, hashtags):
    whatsapp_rules = {
        "WhatsApp Status": "1–2 short emotional or catchy lines.",
        "WhatsApp DM": "Friendly, personal, conversational message.",
        "WhatsApp Announcement": "Formal circular / broadcast announcement."
    }

    platform_rule = whatsapp_rules.get(platform, f"Optimized for {platform}")

    hashtag_rule = (
        "Add 5–10 niche relevant hashtags at the end."
        if hashtags else
        "Do NOT include hashtags."
    )

    length_rule = {
        "Short": "Very concise.",
        "Medium": "Balanced length.",
        "Long": "Detailed and descriptive."
    }.get(length, "Medium length")

    return f"""
You are an expert Social Media Strategist.

Platform: {platform}
Tone: {tone}
Goal: {goal}
Language: {language}
Length: {length_rule}

Rules:
- {platform_rule}
- Start with a strong hook
- Use short readable lines
- End with a CTA
- {hashtag_rule}
- If language is Roman Urdu, write ONLY Roman Urdu
- Output ONLY the final caption/message
"""

# ---------------- CHAT LOGIC ----------------
def generate_or_refine(
    topic,
    refine_input,
    platform,
    tone,
    length,
    goal,
    language,
    hashtags,
    history
):
    if not history and not topic.strip():
        return history, history, ""

    sys_msg = build_system_prompt(platform, tone, language, goal, length, hashtags)

    messages = [{"role": "system", "content": sys_msg}]

    for msg in history:
        messages.append(msg)

    if not history:
        user_text = f"TOPIC: {topic}"
    else:
        user_text = refine_input

    messages.append({"role": "user", "content": user_text})

    try:
        res = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=0.7,
            max_tokens=900
        )
        reply = res.choices[0].message.content
    except Exception as e:
        reply = f"❌ Error: {e}"

    history.append({"role": "user", "content": user_text})
    history.append({"role": "assistant", "content": reply})

    return history, history, ""

# ---------------- CLEAR ----------------
def clear_chat():
    return [], [], ""

# ---------------- UI ----------------
css = """
#gen {background: linear-gradient(90deg,#ff512f,#dd2476); color:white; font-weight:bold;}
#clear {background:#444; color:white;}
footer {visibility:hidden;}
"""

with gr.Blocks(css=css) as app:
    gr.Markdown("## 🤖 AI Social Media Manager (Pro Chat Edition)")
    gr.Markdown("Generate captions → refine like ChatGPT → platform optimized")

    chat_state = gr.State([])

    with gr.Row():
        with gr.Column(scale=1):
            platform = gr.Dropdown(
                ["Instagram","LinkedIn","Twitter (X)",
                 "WhatsApp Status","WhatsApp DM","WhatsApp Announcement"],
                value="Instagram",
                label="Platform"
            )
            tone = gr.Dropdown(
                ["Professional","Funny","Emotional","Sales","Witty"],
                value="Professional",
                label="Tone"
            )
            length = gr.Radio(
                ["Short","Medium","Long"],
                value="Medium",
                label="Caption Length"
            )
            goal = gr.Dropdown(
                ["Engagement","Brand Awareness","Sales","Traffic"],
                value="Engagement",
                label="Goal"
            )
            language = gr.Dropdown(
                ["English","Roman Urdu","Hindi"],
                value="English",
                label="Language"
            )
            hashtags = gr.Checkbox(label="Include Hashtags", value=True)

            topic = gr.Textbox(
                label="Topic / Caption Idea",
                placeholder="e.g. Remote work productivity",
                lines=3
            )

            refine_input = gr.Textbox(
                label="Refine / Edit (Chat)",
                placeholder="e.g. Isko short karo, funny bana do",
                lines=3
            )

            with gr.Row():
                gen_btn = gr.Button("✨ Generate / Refine", elem_id="gen")
                clear_btn = gr.Button("🧹 Clear", elem_id="clear")

        with gr.Column(scale=1):
            chatbot = gr.Chatbot(
                label="AI Chat",
                height=480,
                type="messages"
            )

    gen_btn.click(
        generate_or_refine,
        inputs=[topic, refine_input, platform, tone, length, goal, language, hashtags, chat_state],
        outputs=[chatbot, chat_state, refine_input]
    )

    clear_btn.click(
        clear_chat,
        outputs=[chatbot, chat_state, refine_input]
    )

app.launch(debug=True)

/tmp/ipython-input-892552012.py:122: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css) as app:
/tmp/ipython-input-892552012.py:175: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e3d734c3e7eb86c71f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e3d734c3e7eb86c71f.gradio.live


In [6]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 9.2 MB/s eta 0:00:00
